# Hyper parameters optimization #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
repeats = 5
n_trials = 10
importance_type = "split"
#first round of optimization
lgb_params = {"bagging_fraction": 0.9522534844058304, 
              "boosting_type": "dart", 
              "objective": "regression",
              "feature_fraction": 0.42236910941558053, 
              "lambda_l1": 0.020847266580277746, 
              "lambda_l2": 2.8448564854773326, 
              "learning_rate": 0.11484015430016059, 
              "max_depth": 3, 
              "max_leaves": 35, 
              "min_data_in_leaf": 9,
              "num_iterations": 150
             }
life_history = ["lifespan", "mass_g", "mtGC", "metabolic_rate", "temperature", "gestation_days"]
trait = "lifespan"

debug_local = True #to use local version

In [63]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '..', '..', '..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [65]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.selection import ShapSelector
from yspecies.tuning import Tune
from yspecies.models import ResultsCV, CrossValidator
from yspecies.results import FeatureSummary
import optuna
from optuna import Study, Trial

In [66]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [67]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [68]:
default_selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
default_selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,lifespan,[]


In [69]:
loader = DataLoader(locations, default_selection)
selections = loader.load_life_history()
selections[trait][0]

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(414, 12337)","(12337, 40)",41,414,"(12337, 2)","(41, 18)"


## Setting up ShapSelector ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [9]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)

## Setting up features to select ##

In [10]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type = "split"
)

In [11]:
select_lifespan = replace(selection, select_by = "shap")

In [70]:
url = f'sqlite:///' +str((locations.interim.trials).absolute())
print('loading (if exists) study from '+url)
storage = optuna.storages.RDBStorage(
    url=url
    #engine_kwargs={'check_same_thread': False}
)

study = optuna.multi_objective.study.create_study(directions=['maximize','minimize','maximize'], storage = storage, study_name = f"{trait}_r2_huber_kendall", load_if_exists = True)

loading (if exists) study from sqlite:////data/sources/yspecies/notebooks/../data/interim/trials.sqlite


create_study is experimental (supported from v1.4.0). The interface can change in the future.
NSGAIIMultiObjectiveSampler is experimental (supported from v1.5.0). The interface can change in the future.
RandomMultiObjectiveSampler is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-23 11:08:52,478] Using an existing study with name 'lifespan_r2_huber_kendall' instead of creating a new one.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.


In [13]:
def objective_parameters(trial: Trial) -> dict:
    return {
        'objective': 'regression',
        'metric': {'mae', 'mse', 'huber'},
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['dart', 'gbdt']),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.01, 3.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.01, 3.0),
        'max_leaves': trial.suggest_int("max_leaves", 15, 25),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.3, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 1.0),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 3, 8),
        'drop_rate': trial.suggest_uniform('drop_rate', 0.1, 0.3),
        "verbose": -1
    }
optimization_parameters = objective_parameters

In [14]:
from yspecies.workflow import SplitReduce

def side(i: int):
    print(i)
    return i

prepare_partition = SplitReduce(
    outputs = DataPartitioner(), 
    split = lambda x: [(x[0], replace(partition_params, seed=side(x[2])))], 
    reduce = lambda x, output: (output[0], x[1]) 
)                               
partition_and_cv = Pipeline(
    [
        ("prepare partition", prepare_partition),
        ("shap_computation", ShapSelector()) #('crossvalidator', CrossValidator())        
    ]
)

partition_and_cv_repeat =  Pipeline([
    ("repeat_cv_pipe", Repeat(partition_and_cv, repeats, lambda x, i: [x[0], x[1], i] )),
    #("collect_mean", Collect(fold=lambda results: np.array([r.last("huber") for r in results]).mean()))
    ("collect_mean", Collect(fold=lambda results: (FeatureSummary(results).metrics_average.R2, FeatureSummary(results).metrics_average.huber, FeatureSummary(results).kendall_tau_abs_mean)))    
    ]
    )

p = Pipeline([
     ('extractor', DataExtractor()),
     ('tune', Tune(partition_and_cv_repeat, study = study, n_trials = n_trials, parameters_space = optimization_parameters))    
])

In [15]:
results = p.fit_transform(selections[trait])
results

MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


0
===== fitting models with seed 0 =====
SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Equus_caballus']


Early stopping is not available in dart mode


[150]	valid_0's l2: 258.363	valid_0's l1: 9.88479	valid_0's huber: 8.53768
SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Ursus_americanus', 'Dasypus_novemcinctus']
[150]	valid_0's l2: 168.597	valid_0's l1: 5.98722	valid_0's huber: 5.02284
SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Capra_hircus', 'Mesocricetus_auratus']
[150]	valid_0's l2: 74.7082	valid_0's l1: 4.96547	valid_0's huber: 4.10364
SEED: 0 | FOLD: 3 | VALIDATION_SPECIES: ['Otolemur_garnettii', 'Macaca_fascicularis']
[150]	valid_0's l2: 112.06	valid_0's l1: 4.92143	valid_0's huber: 4.06918
1
===== fitting models with seed 1 =====
SEED: 1 | FOLD: 0 | VALIDATION_SPECIES: ['Rattus_norvegicus', 'Vombatus_ursinus']


Early stopping is not available in dart mode


...

[150]	valid_0's l2: 88.9507	valid_0's l1: 5.98855	valid_0's huber: 5.02668
SEED: 4 | FOLD: 1 | VALIDATION_SPECIES: ['Otolemur_garnettii', 'Ailuropoda_melanoleuca']
[150]	valid_0's l2: 111.677	valid_0's l1: 5.16135	valid_0's huber: 4.30619
SEED: 4 | FOLD: 2 | VALIDATION_SPECIES: ['Ursus_americanus', 'Oryctolagus_cuniculus']
[150]	valid_0's l2: 87.4141	valid_0's l1: 5.51997	valid_0's huber: 4.61481
SEED: 4 | FOLD: 3 | VALIDATION_SPECIES: ['Mesocricetus_auratus', 'Mus_spicilegus']
[150]	valid_0's l2: 67.6427	valid_0's l1: 4.06962	valid_0's huber: 3.30577


MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-23 08:35:02,520] Trial 23 finished with values: [0.8543807146219302, 4.657504235631242, 0.4914909173317466] with parameters: {'boosting_type': 'dart', 'lambda_l1': 1.795082198936093, 'lambda_l2': 0.19612051897858127, 'max_leaves': 20, 'max_depth': 3, 'feature_fraction': 0.3108481512627257, 'bagging_fraction': 0.7887056432452557, 'learning_rate': 0.09534666251419048, 'min_data_in_leaf': 8, 'drop_rate': 0.1979587993448328}.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
FrozenMultiObjectiveTrial is experimental (supported

0
===== fitting models with seed 0 =====
SEED: 0 | FOLD: 0 | VALIDATION_SPECIES: ['Gorilla_gorilla', 'Equus_caballus']
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[85]	valid_0's l2: 240.061	valid_0's l1: 8.24137	valid_0's huber: 7.09437
SEED: 0 | FOLD: 1 | VALIDATION_SPECIES: ['Ursus_americanus', 'Dasypus_novemcinctus']
Training until validation scores don't improve for 10 rounds
[150]	valid_0's l2: 151.206	valid_0's l1: 4.55677	valid_0's huber: 3.81682
Did not meet early stopping. Best iteration is:
[150]	valid_0's l2: 151.206	valid_0's l1: 4.55677	valid_0's huber: 3.81682
SEED: 0 | FOLD: 2 | VALIDATION_SPECIES: ['Capra_hircus', 'Mesocricetus_auratus']
Training until validation scores don't improve for 10 rounds
[150]	valid_0's l2: 76.1951	valid_0's l1: 4.852	valid_0's huber: 4.03678
Did not meet early stopping. Best iteration is:
[150]	valid_0's l2: 76.1951	valid_0's l1: 4.852	valid_0's huber: 4.03678
SEED: 0 | FOLD: 3 | VALIDATION_

MultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.
MultiObjectiveStudy is experimental (supported from v1.4.0). The interface can change in the future.
[I 2020-08-23 08:38:49,425] Trial 24 finished with values: [0.8466441128640181, 3.9292641754153634, 0.4287841062841614] with parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.9021473515039649, 'lambda_l2': 2.9324057456181545, 'max_leaves': 21, 'max_depth': 7, 'feature_fraction': 0.6371450345693939, 'bagging_fraction': 0.6860874890763271, 'learning_rate': 0.08299588210084567, 'min_data_in_leaf': 4, 'drop_rate': 0.2426041334113893}.
FrozenMultiObjectiveTrial is experimental (supported from v1.4.0). The interface can change in the future.


MultiObjectiveResults(best_trials=[<optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6bbe20>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6bbee0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c43a0>], all_trials=[<optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f7fdf40>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f75f850>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c5fd0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c6700>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c6760>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c67c0>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c6820>, <optuna.multi_objective.trial.FrozenMultiObjectiveTrial object at 0x7f171f6c6880>, <optuna.multi_objective.trial.FrozenMul

In [33]:
best = results.best_trials

In [72]:
for i, t in enumerate(best):
    trait_path = locations.metrics.optimization / trait
    if not trait_path.exists():
        trait_path.mkdir()
    path = trait_path / f"{str(i)}.json"    
    print(f"writing parameters to {path}")
    with open(path, 'w') as f:
        params = t.params
        values = t.values
        to_write = {"number": t.number,"params": params, "metrics": {"R2":values[0], "huber": values[1], "kendall_tau": values[2]}}
        json.dump(to_write, f, sort_keys=True, indent=4)

writing parameters to ../data/metrics/optimization/lifespan/0.json
writing parameters to ../data/metrics/optimization/lifespan/1.json
writing parameters to ../data/metrics/optimization/lifespan/2.json
